In [155]:
!pip install duckdb

In [156]:
import duckdb

In [157]:
con = duckdb.connect(database=':memory:')

In [158]:
# must be a table to use INSERT afterwards
con.execute("drop table if exists input").execute(
    "CREATE TABLE input AS "
    "SELECT id as rid, concat(given_name, ' ', surname, ' ', date_of_birth) as val "
    "FROM 'data/S1_clean_.csv'")

In [159]:
con.execute(
    "insert into input "
    "select id, concat(given_name, ' ', surname, ' ', date_of_birth) "
    "from 'data/S2_clean_.csv'")

In [160]:
con.execute(
    "insert into input "
    "select id, concat(given_name, ' ', surname, ' ', date_of_birth) "
    "from 'data/S3_clean_.csv'")

In [161]:
con.execute("select * from input").fetchall()

[('S1_0', 'joshua morrison 19101123'),
 ('S1_1', 'jordan white 19371126'),
 ('S1_2', 'emmerson lock 19211129'),
 ('S1_3', 'alexandra grosser 19720305'),
 ('S1_4', 'michael wuchatsch 19190110'),
 ('S1_5', 'emmerson loyck 19211129'),
 ('S1_6', 'rhys schuetz 19440909'),
 ('S1_7', 'joshua greenj 19790110'),
 ('S1_8', 'olivia hobson 19760812'),
 ('S1_9', 'michael lierach 19360816'),
 ('S1_10', 'elisabett domiten 19081008'),
 ('S1_11', 'genoveffa hylander 19071008'),
 ('S2_0', 'braecon schuetz 19440909'),
 ('S2_1', 'alexandra grosvenor 19930305'),
 ('S2_2', 'michael liersch 19360816'),
 ('S2_3', 'emmeron loyk 19321129'),
 ('S2_4', 'olivia hobson 19760812'),
 ('S2_5', 'joshua green 19010219'),
 ('S2_6', 'charlotte hyland 19340909'),
 ('S2_7', 'elisabet domitienn 19071008'),
 ('S3_0', 'emmerson loyck 19211129'),
 ('S3_1', 'michel wuchatsch 19190110'),
 ('S3_3', 'liersch michael 19360816'),
 ('S3_4', 'charlotte hyland 19460401'),
 ('S3_5', 'braedon schuetz 19440909'),
 ('S3_6', 'olivia hobson 1

In [162]:
# Words tokenizer, possible option for the incoming library
# not splitting on single quote (') yet
import string
con.execute(
    "select rid, len(tks) as rlen, unnest(tks) as token "
    "from ( "
    r"""select distinct rid, str_split_regex(val, '[!"#$%&()*+,-./:;<=>?@[\]^_`{|}~""" + string.whitespace + """]') as tks """
    """from input """
    ") "
).fetchall()

[('S1_0', 3, 'joshua'),
 ('S1_0', 3, 'morrison'),
 ('S1_0', 3, '19101123'),
 ('S1_1', 3, 'jordan'),
 ('S1_1', 3, 'white'),
 ('S1_1', 3, '19371126'),
 ('S1_2', 3, 'emmerson'),
 ('S1_2', 3, 'lock'),
 ('S1_2', 3, '19211129'),
 ('S1_3', 3, 'alexandra'),
 ('S1_3', 3, 'grosser'),
 ('S1_3', 3, '19720305'),
 ('S1_4', 3, 'michael'),
 ('S1_4', 3, 'wuchatsch'),
 ('S1_4', 3, '19190110'),
 ('S1_5', 3, 'emmerson'),
 ('S1_5', 3, 'loyck'),
 ('S1_5', 3, '19211129'),
 ('S1_6', 3, 'rhys'),
 ('S1_6', 3, 'schuetz'),
 ('S1_6', 3, '19440909'),
 ('S1_7', 3, 'joshua'),
 ('S1_7', 3, 'greenj'),
 ('S1_7', 3, '19790110'),
 ('S1_8', 3, 'olivia'),
 ('S1_8', 3, 'hobson'),
 ('S1_8', 3, '19760812'),
 ('S1_9', 3, 'michael'),
 ('S1_9', 3, 'lierach'),
 ('S1_9', 3, '19360816'),
 ('S1_10', 3, 'elisabett'),
 ('S1_10', 3, 'domiten'),
 ('S1_10', 3, '19081008'),
 ('S1_11', 3, 'genoveffa'),
 ('S1_11', 3, 'hylander'),
 ('S1_11', 3, '19071008'),
 ('S2_0', 3, 'braecon'),
 ('S2_0', 3, 'schuetz'),
 ('S2_0', 3, '19440909'),
 ('S2_1', 

In [163]:
q = 3

In [164]:
maxlen = con.execute("select max(length(val)) from input").fetchall()[0][0]

In [165]:
con.execute("drop view if exists tokens").execute(
    "create view tokens as "
    f"select distinct rid, length(val) + {q} - 1 as rlen "
    f", substring(concat(repeat('#', {q} - 1), "
    "lower(val), "
    f"repeat('#',{q} - 1)),"
    f"x, {q}) as token "
    f"from input, range(1, {maxlen + q}) tbl(x) "
    f"where x <= length(val) + {q} - 1"
)

In [166]:
con.execute("select * from tokens").fetchall()

[('S2_1', 30, '5##'),
 ('S2_1', 30, '05#'),
 ('S1_11', 29, '8##'),
 ('S2_7', 29, '8##'),
 ('S2_1', 30, '305'),
 ('S1_11', 29, '08#'),
 ('S2_7', 29, '08#'),
 ('S1_3', 28, '5##'),
 ('S1_4', 28, '0##'),
 ('S1_10', 28, '8##'),
 ('S3_11', 28, '8##'),
 ('S2_1', 30, '030'),
 ('S1_11', 29, '008'),
 ('S2_7', 29, '008'),
 ('S1_3', 28, '05#'),
 ('S1_4', 28, '10#'),
 ('S1_10', 28, '08#'),
 ('S3_11', 28, '08#'),
 ('S2_6', 27, '9##'),
 ('S3_1', 27, '0##'),
 ('S3_4', 27, '1##'),
 ('S2_1', 30, '303'),
 ('S1_11', 29, '100'),
 ('S2_7', 29, '100'),
 ('S1_3', 28, '305'),
 ('S1_4', 28, '110'),
 ('S1_10', 28, '008'),
 ('S2_6', 27, '09#'),
 ('S3_1', 27, '10#'),
 ('S3_4', 27, '01#'),
 ('S1_0', 26, '3##'),
 ('S1_9', 26, '6##'),
 ('S2_0', 26, '9##'),
 ('S2_2', 26, '6##'),
 ('S3_3', 26, '6##'),
 ('S3_5', 26, '9##'),
 ('S3_9', 26, '3##'),
 ('S2_1', 30, '930'),
 ('S1_11', 29, '710'),
 ('S2_7', 29, '710'),
 ('S1_3', 28, '030'),
 ('S1_4', 28, '011'),
 ('S1_10', 28, '100'),
 ('S2_6', 27, '909'),
 ('S3_1', 27, '110'),

In [167]:
t = 0.5

In [168]:
# brute force join, to get the gold standard
con.execute("drop view if exists bfjoin").execute(
    "create view bfjoin as "
    "select r1.rid as rid1, r2.rid as rid2 "
    "from tokens as r1, tokens as r2 "
    "where r1.token = r2.token "
    "and r1.rid < r2.rid "
    "group by r1.rid, r1.rlen, r2.rid, r2.rlen "
    f"having count(*) >= ceil({t} / (1+{t}) * (r1.rlen + r2.rlen))"
)

In [169]:
con.execute("select * from bfjoin").fetchall()

[('S1_11', 'S3_11'),
 ('S1_0', 'S3_9'),
 ('S1_9', 'S3_3'),
 ('S2_2', 'S3_3'),
 ('S1_5', 'S2_3'),
 ('S1_8', 'S3_6'),
 ('S2_4', 'S3_6'),
 ('S1_4', 'S3_1'),
 ('S1_7', 'S3_7'),
 ('S1_6', 'S2_0'),
 ('S2_6', 'S3_4'),
 ('S1_10', 'S2_7'),
 ('S2_5', 'S3_7'),
 ('S1_9', 'S2_2'),
 ('S2_0', 'S3_5'),
 ('S1_2', 'S1_5'),
 ('S1_6', 'S3_5'),
 ('S1_5', 'S3_0'),
 ('S1_3', 'S2_1'),
 ('S1_8', 'S2_4'),
 ('S1_2', 'S3_0'),
 ('S2_3', 'S3_0')]

## PrefixR

In [170]:
con.execute("drop view if exists doc_frequency").execute(
    "CREATE VIEW doc_frequency AS "
    "SELECT token, count(*) AS df "
    "FROM tokens "
    "GROUP BY token "
    # ORDER BY later!
)

In [171]:
con.execute("SELECT * FROM doc_frequency").fetchall()

[('5##', 2),
 ('05#', 2),
 ('8##', 4),
 ('305', 2),
 ('08#', 4),
 ('0##', 5),
 ('030', 2),
 ('008', 4),
 ('10#', 5),
 ('9##', 9),
 ('1##', 1),
 ('303', 1),
 ('100', 4),
 ('110', 4),
 ('09#', 4),
 ('01#', 1),
 ('3##', 2),
 ('6##', 4),
 ('930', 1),
 ('710', 3),
 ('011', 6),
 ('909', 4),
 ('401', 1),
 ('23#', 2),
 ('16#', 3),
 ('993', 1),
 ('071', 3),
 ('203', 1),
 ('901', 5),
 ('810', 1),
 ('090', 4),
 ('040', 1),
 ('123', 2),
 ('816', 3),
 ('29#', 4),
 ('2##', 3),
 ('199', 1),
 ('907', 3),
 ('720', 1),
 ('190', 8),
 ('081', 7),
 ('409', 4),
 ('604', 1),
 ('112', 7),
 ('129', 4),
 ('12#', 3),
 ('972', 1),
 ('919', 2),
 ('908', 1),
 ('340', 1),
 ('460', 1),
 ('608', 6),
 ('812', 3),
 ('26#', 1),
 ('19#', 1),
 ('r 1', 3),
 ('197', 6),
 ('191', 4),
 ('946', 1),
 ('101', 2),
 ('360', 3),
 ('440', 3),
 ('111', 3),
 ('126', 1),
 ('219', 1),
 ('410', 1),
 ('or ', 1),
 ('193', 6),
 ('194', 4),
 ('910', 2),
 ('936', 3),
 ('944', 3),
 ('211', 4),
 ('021', 1),
 ('041', 1),
 ('nor', 1),
 ('er ', 2),

In [172]:
con.execute("drop view if exists R").execute(
    "CREATE VIEW R AS ("
    "select rid, rlen, token "
    ", row_number() OVER (PARTITION BY rid ORDER BY df, token) as pos "
    "from ("
    "SELECT tt.*, df.df "
    "FROM tokens AS tt "
    "JOIN doc_frequency AS df ON tt.token = df.token "
    ")"
    ")"
)

In [173]:
x = con.execute("select * from R").fetchall()
len(x), x

(770,
 [('S1_0', 26, 'iso', 1),
  ('S1_0', 26, 'ris', 2),
  ('S1_0', 26, ' mo', 3),
  ('S1_0', 26, '101', 4),
  ('S1_0', 26, '123', 5),
  ('S1_0', 26, '23#', 6),
  ('S1_0', 26, '3##', 7),
  ('S1_0', 26, '910', 8),
  ('S1_0', 26, 'a m', 9),
  ('S1_0', 26, 'mor', 10),
  ('S1_0', 26, 'orr', 11),
  ('S1_0', 26, 'rri', 12),
  ('S1_0', 26, '191', 13),
  ('S1_0', 26, 'hua', 14),
  ('S1_0', 26, 'jos', 15),
  ('S1_0', 26, 'osh', 16),
  ('S1_0', 26, 'shu', 17),
  ('S1_0', 26, 'ua ', 18),
  ('S1_0', 26, '##j', 19),
  ('S1_0', 26, '#jo', 20),
  ('S1_0', 26, '011', 21),
  ('S1_0', 26, '112', 22),
  ('S1_0', 26, 'son', 23),
  ('S1_0', 26, 'n 1', 24),
  ('S1_0', 26, 'on ', 25),
  ('S1_0', 26, ' 19', 26),
  ('S1_1', 23, ' wh', 1),
  ('S1_1', 23, '126', 2),
  ('S1_1', 23, '26#', 3),
  ('S1_1', 23, '371', 4),
  ('S1_1', 23, '711', 5),
  ('S1_1', 23, '937', 6),
  ('S1_1', 23, 'an ', 7),
  ('S1_1', 23, 'dan', 8),
  ('S1_1', 23, 'hit', 9),
  ('S1_1', 23, 'jor', 10),
  ('S1_1', 23, 'n w', 11),
  ('S1_1', 23

In [174]:
con.execute("drop view if exists PrefixR").execute(
    "create view PrefixR as "
    "select rid, rlen, token, pos "
    "from R "
    f"where rlen - pos + 1 >= ceil(rlen * {t}) "
)

In [175]:
x = con.execute("select * from PrefixR").fetchall()
len(x), x

(409,
 [('S1_0', 26, 'iso', 1),
  ('S1_0', 26, 'ris', 2),
  ('S1_0', 26, ' mo', 3),
  ('S1_0', 26, '101', 4),
  ('S1_0', 26, '123', 5),
  ('S1_0', 26, '23#', 6),
  ('S1_0', 26, '3##', 7),
  ('S1_0', 26, '910', 8),
  ('S1_0', 26, 'a m', 9),
  ('S1_0', 26, 'mor', 10),
  ('S1_0', 26, 'orr', 11),
  ('S1_0', 26, 'rri', 12),
  ('S1_0', 26, '191', 13),
  ('S1_0', 26, 'hua', 14),
  ('S1_1', 23, ' wh', 1),
  ('S1_1', 23, '126', 2),
  ('S1_1', 23, '26#', 3),
  ('S1_1', 23, '371', 4),
  ('S1_1', 23, '711', 5),
  ('S1_1', 23, '937', 6),
  ('S1_1', 23, 'an ', 7),
  ('S1_1', 23, 'dan', 8),
  ('S1_1', 23, 'hit', 9),
  ('S1_1', 23, 'jor', 10),
  ('S1_1', 23, 'n w', 11),
  ('S1_1', 23, 'ord', 12),
  ('S1_10', 28, '810', 1),
  ('S1_10', 28, '908', 2),
  ('S1_10', 28, 'ett', 3),
  ('S1_10', 28, 'ten', 4),
  ('S1_10', 28, 'tt ', 5),
  ('S1_10', 28, ' do', 6),
  ('S1_10', 28, '#el', 7),
  ('S1_10', 28, 'abe', 8),
  ('S1_10', 28, 'bet', 9),
  ('S1_10', 28, 'dom', 10),
  ('S1_10', 28, 'eli', 11),
  ('S1_10',

In [176]:
con.execute("drop view if exists candset").execute(
    "CREATE VIEW candset AS ("
    "SELECT R1.rid AS rid1, R2.rid AS rid2 "
    ", MAX(R1.pos) as maxPos1, MAX(R2.pos) as maxPos2, count(*) as prOverlap "
    "FROM PrefixR R1, PrefixR R2 "
    "WHERE R1.rid < R2.rid "
    "AND R1.token = R2.token "
    # length filter
    f"AND R1.rlen >= ceil({t} * R2.rlen)"
    # prefix filter
    f"AND R1.rlen - R1.pos + 1 >= CEIL(R1.rlen * 2 * {t} / (1+{t})) "
    # positional filter
    "AND LEAST((R1.rlen - R1.pos + 1), (R2.rlen - R2.pos + 1)) >= "
    f"CEIL((R1.rlen + R2.rlen) * {t} / (1 + {t})) "
    "GROUP BY R1.rid, R2.rid "
    ")"
)

In [177]:
con.execute("select * from candset").fetchall()

[('S1_0', 'S3_9', 7, 9, 5),
 ('S1_10', 'S2_7', 10, 11, 5),
 ('S1_11', 'S3_11', 10, 10, 6),
 ('S1_2', 'S3_0', 8, 9, 6),
 ('S1_3', 'S2_1', 8, 11, 2),
 ('S1_4', 'S3_1', 7, 9, 7),
 ('S1_5', 'S3_0', 9, 9, 9),
 ('S1_7', 'S3_7', 9, 7, 6),
 ('S1_8', 'S3_6', 9, 9, 9),
 ('S1_9', 'S2_2', 9, 7, 6),
 ('S1_9', 'S3_3', 8, 9, 3),
 ('S2_0', 'S3_5', 9, 9, 6),
 ('S2_2', 'S3_3', 6, 9, 4),
 ('S2_3', 'S3_0', 8, 9, 2),
 ('S2_4', 'S3_6', 9, 9, 9),
 ('S2_5', 'S3_7', 8, 5, 3),
 ('S2_6', 'S3_4', 5, 10, 3),
 ('S1_2', 'S1_5', 8, 9, 6),
 ('S1_5', 'S2_3', 9, 8, 2),
 ('S1_6', 'S3_5', 7, 9, 1),
 ('S1_7', 'S2_5', 7, 8, 2),
 ('S1_8', 'S2_4', 9, 9, 9),
 ('S1_2', 'S2_3', 8, 8, 1),
 ('S1_6', 'S2_0', 7, 9, 1)]

In [178]:
con.execute("drop view if exists matches").execute(
    "create view matches as "
    "select r1.rid as rid1, r2.rid as rid2 "
    "from R r1, R r2, candset c "
    "where c.rid1 = r1.rid "
    "and c.rid2 = r2.rid "
    "and r1.token = r2.token "
    "and r1.pos > maxPos1 "
    "and r2.pos > maxPos2 "
    "group by r1.rid, r2.rid, r1.rlen, r2.rlen, prOverlap "
    f"having count(*) + prOverlap >= (r1.rlen + r2.rlen) * {t} / (1+{t})"
)

In [179]:
con.execute("select * from matches").fetchall()

[('S1_0', 'S3_9'),
 ('S1_10', 'S2_7'),
 ('S1_11', 'S3_11'),
 ('S1_2', 'S3_0'),
 ('S1_3', 'S2_1'),
 ('S1_4', 'S3_1'),
 ('S1_5', 'S3_0'),
 ('S1_6', 'S3_5'),
 ('S1_7', 'S3_7'),
 ('S1_8', 'S3_6'),
 ('S1_9', 'S3_3'),
 ('S1_9', 'S2_2'),
 ('S2_0', 'S3_5'),
 ('S2_2', 'S3_3'),
 ('S2_3', 'S3_0'),
 ('S2_4', 'S3_6'),
 ('S2_5', 'S3_7'),
 ('S2_6', 'S3_4'),
 ('S1_2', 'S1_5'),
 ('S1_5', 'S2_3'),
 ('S1_6', 'S2_0'),
 ('S1_8', 'S2_4')]

In [180]:
con.execute(
    "select i1.val, i2.val "
    # ", i1.rid, i2.rid "
    "from matches m "
    "join input i1 on i1.rid = m.rid1 "
    "join input i2 on i2.rid = m.rid2"
).fetchall()

[('joshua morrison 19101123', 'joshua morriosn 19101123'),
 ('genoveffa hylander 19071008', 'genovefa hyllande 19071008'),
 ('emmerson lock 19211129', 'emmerson loyck 19211129'),
 ('michael wuchatsch 19190110', 'michel wuchatsch 19190110'),
 ('emmerson loyck 19211129', 'emmerson loyck 19211129'),
 ('rhys schuetz 19440909', 'braedon schuetz 19440909'),
 ('joshua greenj 19790110', 'joshua green 19790110'),
 ('olivia hobson 19760812', 'olivia hobson 19760812'),
 ('michael lierach 19360816', 'liersch michael 19360816'),
 ('michael lierach 19360816', 'michael liersch 19360816'),
 ('braecon schuetz 19440909', 'braedon schuetz 19440909'),
 ('michael liersch 19360816', 'liersch michael 19360816'),
 ('emmeron loyk 19321129', 'emmerson loyck 19211129'),
 ('olivia hobson 19760812', 'olivia hobson 19760812'),
 ('joshua green 19010219', 'joshua green 19790110'),
 ('charlotte hyland 19340909', 'charlotte hyland 19460401'),
 ('emmerson lock 19211129', 'emmerson loyck 19211129'),
 ('emmerson loyck 192

# Debug

In [181]:
con.execute(
    "select * "
    "from matches m "
    "full outer join bfjoin b on b.rid1 = m.rid1 and b.rid2 = m.rid2 "
    "where m.rid1 is null "
).fetchall()

[]